# KNN_SVM_프로모션_효율예측분석

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [3]:
mem = pd.read_csv("./data/member.csv")
tran = pd.read_csv("./data/transaction.csv")

In [5]:
mem.head(2)

,id,recency,zip_code,is_referral,channel,conversion
0,906145,10,Surburban,0,Phone,0
1,184478,6,Rural,1,Web,0


In [6]:
tran.head(2)

,id,num_item,total_amount
0,906145,5,34000
1,906145,1,27000


* 쇼핑몰 고객 데이터, 프로모션 쿠폰을 발행하고 사용 여부 데이터 수집
* mem: 고객 id, 최근 방문일, 사는 지역, 추천여부, 주요접속채널, 쿠폰사용여부(target)
* tran: 고객id, 구매수량, 총 구매금액
* 전통적 마케팅 분석 방법 RFM 기법을 활용해 고객 데이터에서 파생변수 생성 후 분석
* R: Recency: 현재일 - 최근 구매일
* F: Frequency: 구매빈도
* M: Monetary : 구매 금액
* 종속변수: coversion => 고객이 프로모션에 반응 했는가? 1 = yes, 0 = no

# 결측값 탐지

In [ ]:
mem.isna().sum()

In [ ]:
tran.isna().sum()

In [ ]:
mem.info()

In [ ]:
tran.info()

In [ ]:
mem.describe()

In [ ]:
tran.describe()

In [ ]:
mem['id'].nunique()

In [ ]:
mem['recency'].value_counts()

In [ ]:
mem['zip_code'].value_counts()


In [ ]:
mem['is_referral'].value_counts()

In [ ]:
mem['channel'].value_counts()


In [ ]:
mem['conversion'].value_counts()

In [ ]:
mem.columns

In [ ]:
tran

In [ ]:
mean_item_amount = tran.groupby('id')[['num_item','total_amount']].mean()
mean_item_amount = mean_item_amount.reset_index()
mean_item_amount.columns = ['id', 'mean_num_item', 'mean_total_amount']
mean_item_amount

In [ ]:
freq = tran.groupby('id')[['id']].count()
freq


In [ ]:
mean_item_amount = mean_item_amount.set_index('id')
mean_item_amount


In [ ]:
mean_item_amount = mean_item_amount.join(freq)
mean_item_amount


In [ ]:
mean_item_amount = mean_item_amount.rename(columns={'id':'frequency'})
mean_item_amount

총구매 개수, 총 금액 추가

In [ ]:
total_num_amount = tran.groupby('id')[['num_item', 'total_amount']].sum()
total_num_amount

In [ ]:
mean_item_amount = mean_item_amount.join(total_num_amount)
mean_item_amount


In [ ]:
mean_item_amount = mean_item_amount.reset_index()
mean_item_amount

회원정보 테이블과 구매갯수, 금액 등을 그룹연산한 mean_item_amount 합치기

In [ ]:
data = pd.merge(mem, mean_item_amount, how='left', on='id')
data

In [ ]:
data.columns


In [ ]:


data['mean_num_item'].plot(kind='hist')

In [ ]:
data['frequency'].plot(kind='hist')


In [ ]:
data['num_item'].plot(kind='hist')

In [ ]:


data['total_amount'].plot(kind='hist')


In [ ]:
col_names = ['recency', 'zip_code', 'is_referral', 'channel', 
       'mean_num_item', 'mean_total_amount', 'frequency', 'num_item',
       'total_amount']


In [ ]:
for col in col_names:
    print("="*30, col, "="*30)
    print(data.groupby(col)['conversion'].mean().sort_values(ascending=False))
    print()

# 거리기반의 알고리즘을 사용시 독립변수들 간의 단위를 꼭 맞춰 주어야 함
# 스케일링
* MinMaxScaler: 모든 숫자를 0-1 사이의 숫자로 변환 - 데이터 분포의 모양을 그대로 유지
* StandardScaler: 평균을 0, 표준편차를 1로 하는 정규분포 형태로 변환 - 데이터의 분포 모양이 정규 분포로 바뀜
* RobustScaler: 사분위수를 이용해서 데이터를 스케일링 - 데이터에 이상값이 있을 때 사용, 이상값에 영향을 최소화 

# 머신러닝 모델별 스케일러
* knn(최근접이웃): MinMaxScaler, 이상치가 있는 경우 RobustScaler
* SVM(서포트 벡터 머신): StandardScaler, 이상치가 있는 경우 RobustScaler
* Logistic Regression: Standard, 이상치가 있는 경우 RobustScaler
* Linear / Ridge / Lasso: StandardScaler, 이상치가 있는 경우 RobustScaler
* KMeans / DBSCAN: MinMax or Standard, 이상치가 있는 경우 RobustScaler
* DecisionTree, RandomForest, XGBoost: 스케일링 불필요, 이상치가 있는 경우에도 안 해도 됨
* Navie Bayes: 스케일링 불필요,

# 스케일링 시점: train / test로 나눈 후에 실시

In [ ]:
data

In [ ]:
data.columns

In [ ]:
data = pd.get_dummies(data, columns=['zip_code', 'channel'], drop_first=True)

홀드아웃

In [ ]:
X = data.drop(['id', 'conversion'], axis=1)
y = data['conversion']


In [ ]:


from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, stratify=y_valid, random_state=42)



# 데이터 단위를 맞추기 위해서 Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [ ]:
mms = MinMaxScaler()
mms.fit(X_train)
mms_X_train = mms.transform(X_train)
mms_X_valid = mms.transform(X_valid)
mms_X_test = mms.transform(X_test)


In [ ]:


mms_X_train = pd.DataFrame(mms_X_train, columns=X_train.columns)
mms_X_valid = pd.DataFrame(mms_X_valid, columns=X_train.columns)
mms_X_test = pd.DataFrame(mms_X_test, columns=X_train.columns)

In [ ]:
X_test

In [ ]:
mms_X_test

# KNN (K-nearest Neighbor)

In [ ]:


from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:


knn = KNeighborsClassifier(n_jobs=8)
knn.fit(mms_X_train, y_train)
valid_pred = knn.predict(mms_X_valid)
print(classification_report(y_valid, valid_pred))

# KNN의 하이퍼파라미터 튜닝
* n_neighbors = ?
* 전체 샘플 수가 적을 때 10000개 이하일 때 3, 5, 7 같은 홀수 값
* 전체 샘플 수가 10000개 초과시 루트 (n), log2(n)

In [ ]:
len(mms_X_train)

# 최적 K 는?

In [ ]:


# 루트를 이용한 최적 K 계산
n = len(mms_X_train)
print(n)
k = int(np.sqrt(n))
print(np.sqrt(n), k)


In [ ]:
# log2(n)을 이용한 최적 k 계산
n = len(mms_X_train)
k_log2 = int(np.log2(n))
print(k_log2)

In [ ]:
# 루트를 이용해 계산한 k 값 195 적용
knn = KNeighborsClassifier(n_neighbors=195, n_jobs=8)
knn.fit(mms_X_train, y_train)
valid_pred = knn.predict(mms_X_valid)
print(classification_report(y_valid, valid_pred))

In [ ]:
# log2(n)를 이용해 계산한 k 값 15 적용
knn = KNeighborsClassifier(n_neighbors=15, n_jobs=8)
knn.fit(mms_X_train, y_train)
valid_pred = knn.predict(mms_X_valid)
print(classification_report(y_valid, valid_pred))

In [ ]:
y.value_counts()

In [ ]:
mms_X_train.columns

In [ ]:
from imblearn.over_sampling import SMOTENC

In [ ]:
smtnc = SMOTENC(categorical_features=[1, 7, 8, 9, 10], random_state=42)
smt_X_train, smt_y_train = smtnc.fit_resample(mms_X_train, y_train)

In [ ]:
smt_y_train.value_counts()

In [ ]:


len(smt_X_train)

In [ ]:
n = len(smt_X_train)
k_log2 = int(np.log2(n))
print(k_log2)


In [ ]:
# log2(n)를 이용해 계산한 k 값 15 적용
knn = KNeighborsClassifier(n_neighbors=15, n_jobs=8)
knn.fit(smt_X_train, smt_y_train)
valid_pred = knn.predict(mms_X_valid)
print(classification_report(y_valid, valid_pred))
print("========= Test Result=========")
test_pred = knn.predict(X_test)
print(classification_report(y_test, test_pred))


In [ ]:


# log2(n)를 이용해 계산한 k 값 15 적용
for i in range(3, 22, 2):
    knn = KNeighborsClassifier(n_neighbors=i, n_jobs=8)
    knn.fit(smt_X_train, smt_y_train)
    valid_pred = knn.predict(mms_X_valid)
    print(classification_report(y_valid, valid_pred))
    print(f"========= Test Result {i}=========")
    test_pred = knn.predict(X_test)
    print(classification_report(y_test, test_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight="balanced", n_jobs=8, random_state=42)
rfc.fit(smt_X_train, smt_y_train)
valid_pred = rfc.predict(mms_X_valid)
print("========= Valid Result=========")
print(classification_report(y_valid, valid_pred))
print("========= Test Result=========")
test_pred = rfc.predict(X_test)
print(classification_report(y_test, test_pred))
print()



# 서포트 벡터 SVC를 사용해서 분석
* standardScaler
* 카테고리 변수를 제외하고 StandardSclaer를 사용하는 것이 좋음

In [ ]:


from sklearn.preprocessing import StandardScaler

In [ ]:
X_train.columns

In [ ]:


X_train_cat = X_train[['is_referral', 'zip_code_Surburban',
       'zip_code_Urban', 'channel_Phone', 'channel_Web']]
X_train_num = X_train[['recency', 'mean_num_item', 'mean_total_amount',
       'frequency', 'num_item', 'total_amount']]
X_valid_cat = X_valid[['is_referral', 'zip_code_Surburban',
       'zip_code_Urban', 'channel_Phone', 'channel_Web']]
X_valid_num = X_valid[['recency', 'mean_num_item', 'mean_total_amount',
       'frequency', 'num_item', 'total_amount']]
X_test_cat = X_test[['is_referral', 'zip_code_Surburban',
       'zip_code_Urban', 'channel_Phone', 'channel_Web']]
X_test_num = X_test[['recency', 'mean_num_item', 'mean_total_amount',
       'frequency', 'num_item', 'total_amount']]


In [ ]:
X_train_num.columns


In [ ]:
num_cols = X_train_num.columns

In [ ]:
X_valid[num_cols]


In [ ]:


ss = StandardScaler()
ss.fit(X_train_num)
X_train_num_temp = ss.transform(X_train_num)
X_valid_num_temp = ss.transform(X_valid[num_cols])
X_test_num_temp = ss.transform(X_test[num_cols])


In [ ]:
X_train_num.index


In [ ]:
ss_X_train_num = pd.DataFrame(X_train_num_temp, columns=num_cols, index=X_train_num.index)
ss_X_valid_num = pd.DataFrame(X_valid_num_temp, columns=num_cols, index=X_valid.index)
ss_X_test_num = pd.DataFrame(X_test_num_temp, columns=num_cols, index=X_test.index)

In [ ]:
X_train_cat.columns

In [ ]:
cat_cols = X_train_cat.columns

In [ ]:
ss_X_train = pd.concat([ss_X_train_num, X_train_cat], axis=1)

In [ ]:
ss_X_valid = pd.concat([ss_X_valid_num, X_valid[cat_cols]], axis=1)
ss_X_test = pd.concat([ss_X_test_num, X_test[cat_cols]], axis=1)


In [ ]:


from sklearn.svm import SVC

In [ ]:


svc = SVC()
svc.fit(ss_X_train, y_train)
valid_pred = svc.predict(ss_X_valid)
print("========= Valid Result=========")
print(classification_report(y_valid, valid_pred))
print("========= Test Result=========")
test_pred = svc.predict(ss_X_test)
print(classification_report(y_test, test_pred))
print()
